# 0. Importing Necessary Packages

In [1]:
# Printing the information of Python, IPython, OS, and the generation date.
%load_ext version_information
%version_information

Software versions
Python 3.7.9 64bit [GCC 7.3.0]
IPython 7.16.1
OS Linux 5.8.18 100.fc31.x86_64 x86_64 with fedora 31 Thirty_One
Tue Apr 05 17:13:02 2022 KST

In [2]:
# Printing the versions of packages
from importlib_metadata import version
for pkg in ['numpy', 'matplotlib', 'astropy', 'astroalign', 'photutils']:
    print(pkg+": ver "+version(pkg))

numpy: ver 1.21.0
matplotlib: ver 3.2.2
astropy: ver 4.2.1
astroalign: ver 2.4.1
photutils: ver 1.0.2


In [3]:
# matplotlib backend
%matplotlib notebook

# importing necessary modules
import time
import numpy as np
import glob, os
from astropy.io import fits
import astroalign as aa

# 1. Inspecting the Non-aligned Images (Processed)

### Example Data: the processed data of M51a (R-band, 120 sec), 4 frames

In [4]:
dir_img = "M51a/"
imglist = sorted(glob.glob(dir_img+"*_processed.fits"))
n_img = len(imglist)
imglist

['M51a/M51a-0001R_processed.fits',
 'M51a/M51a-0002R_processed.fits',
 'M51a/M51a-0003R_processed.fits',
 'M51a/M51a-0004R_processed.fits']

In [5]:
# You can also run this command in terminal.
ds9_options = "-scalemode zscale -scale lock yes -frame lock image -tile grid mode manual -tile grid layout 4 1 "
names = ""
for i in np.arange(n_img):
    names += imglist[i]+" "
ds9_command = "ds9 "+ds9_options+names+"&"
print('Running "'+ds9_command+'" in the terminal...')
os.system(ds9_command)

Running "ds9 -scalemode zscale -scale lock yes -frame lock image -tile grid mode manual -tile grid layout 4 1 M51a/M51a-0001R_processed.fits M51a/M51a-0002R_processed.fits M51a/M51a-0003R_processed.fits M51a/M51a-0004R_processed.fits &" in the terminal...


0

### In the DS9 viewer, you can check if the images are aligned or not with some example stars in the images.

* **Use crosshair: ``[Edit] - [Crosshair]`` and ``[Frame] - [Lock] - [Crosshair] - [Image]``** 

* **Zoom-in for a single bright star to test the alignment.**

# 2. Aligning Images with Astroalign (Automatic)

In [6]:
# ----- Solving registration with astroalign ----- #
# Reference image: Frame 1
id_ref = 0
dat_ref, hdr_ref = fits.getdata(imglist[id_ref], header=True, ext=0)

# Aligning other images with respect to the reference image
start_time = time.time()
for i in np.arange(n_img):
    dat, hdr = fits.getdata(imglist[i], header=True, ext=0)
    dat = dat.byteswap().newbyteorder()    # ValueError: Big-endian buffer not supported on little-endian compiler
    if (i == id_ref):
        dat_aligned = dat
    else:
        dat_aligned, footprint = aa.register(dat, dat_ref, max_control_points=50,
                                             detection_sigma=5, min_area=16, propagate_mask=True)
    fits.writeto(imglist[i].split(".fit")[0]+"_align_auto.fits", dat_aligned, overwrite=True)
end_time = time.time()
print(f"--- {end_time-start_time:.4f} sec were taken for aligning {n_img:d} images ---")

--- 5.2363 sec were taken for aligning 4 images ---


* **If the results of this automatic method are not satisfactory, then you should change the key parameters such as ``max_control_points``, ``detection_sigma``, ``min_area``, etc.**

* **In particular, increasing ``min_area`` can improve the results of aligning.**

In [7]:
# You can also run this command in terminal.
ds9_options = "-scalemode zscale -scale lock yes -frame lock image -tile grid mode manual -tile grid layout 4 1 "
names = ""
for i in np.arange(n_img):
    names += imglist[i].split(".fit")[0]+"_align_auto.fits "
ds9_command = "ds9 "+ds9_options+names+"&"
print('Running "'+ds9_command+'" in the terminal...')
os.system(ds9_command)

Running "ds9 -scalemode zscale -scale lock yes -frame lock image -tile grid mode manual -tile grid layout 4 1 M51a/M51a-0001R_processed_align_auto.fits M51a/M51a-0002R_processed_align_auto.fits M51a/M51a-0003R_processed_align_auto.fits M51a/M51a-0004R_processed_align_auto.fits &" in the terminal...


0

### Now the images are well-aligned so we can combine all the science frames.

* **Note that this aligning method is very simple and automatic, so the results might not be so satisfactory. There is a more robust way to do this using the catalogs of detected point sources in the images, but it is beyond the scope of this example code.**

# 3. Combining the Images

In [8]:
img_array = np.zeros((n_img, dat.shape[0], dat.shape[1]))
for i in np.arange(n_img):
    dat_aligned = fits.getdata(imglist[i].split(".fit")[0]+"_align_auto.fits", header=False, ext=0)
    img_array[i, :, :] = dat_aligned
img_combined = np.median(img_array, axis=0)
fits.writeto(dir_img+"M51a_R.fits", img_combined, overwrite=True)